In [2]:
from google import genai
import pandas as pd
from google.genai import types
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_node import ToolNode
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
# from Pydantic import BaseModel, Field
from langchain.pydantic_v1 import BaseModel, Field
from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import MemorySaver
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import openai
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

import yfinance as yf
import pandas as pd
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import initialize_agent
from langchain.schema import HumanMessage, SystemMessage


/Users/zhuoweiyi/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3550: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ.get("open_api")


In [17]:
class tool_for_model():
    @tool
    def calculate_area(length: float, width: float) -> float:
        """Calculate the area of a rectangle given its length and width."""
        return length * width
    @tool
    def fetch_data(interval, period, ticker_symbol):
        """Fetch historical stock data from Yahoo Finance."""
        try:
            data = yf.download(tickers=ticker_symbol, period=period, interval=interval)
        except Exception as e:
            print("Error Occurred:", str(e))
            data = pd.DataFrame()
            data.columns = ['Date','Close', 'High', 'Low', 'Open','Volume']
        return data


In [18]:
llm = init_chat_model("gpt-4.1-nano",model_provider="openai")
llm_with_tools = llm.bind_tools([tool_for_model.calculate_area, tool_for_model.fetch_data])

In [38]:

def generate_strategy(prompt):
    llm = init_chat_model("gpt-4.1-nano",model_provider="openai")

    messages = [
    SystemMessage(content="You are a finaincal and coding master. please generate a strategy for the user, and provide code only using python"),
    HumanMessage(content=prompt),
]
    response = llm.invoke(messages)
    # print(response)
    if len(response.tool_calls) != 0:
        tool_fn = getattr(tool_for_model, response.tool_calls[0]["name"])
        result = tool_fn(**response.tool_calls[0]["args"])
        # print(result)
        return result
    else:
        return response.content

In [20]:
def get_data(prompt):
    messages = [
    SystemMessage(content="You are a finaincal and coding master. please get the data that your stretgy need"),
    HumanMessage(content=prompt),
]
    response = llm_with_tools.invoke(messages)
    # print(response)
    if len(response.tool_calls) != 0:
        tool_fn = getattr(tool_for_model, response.tool_calls[0]["name"])
        result = tool_fn(**response.tool_calls[0]["args"])
        # print(result)
        return result
    else:
        return response.content

In [21]:
def run_function(prompt):
    response = llm_with_tools.invoke("You are a finaincal and coding master. please answer the following questions" + prompt)
    # print(response)
    if len(response.tool_calls) != 0:
        tool_fn = getattr(tool_for_model, response.tool_calls[0]["name"])
        result = tool_fn(**response.tool_calls[0]["args"])
        # print(result)
        return result
    else:
        return response.content

In [45]:
response = generate_strategy("generate a strategy for tsla using yfinance,and provide the stock id at the beginning of the code")

In [55]:
response

'```python\n# Stock ID for Tesla\nticker = "TSLA"\n\nimport yfinance as yf\nimport pandas as pd\nimport numpy as np\n\n# Fetch historical data for TSLA\ndata = yf.download(ticker, period="1y", interval="1d")\n\n# Calculate Moving Averages\ndata[\'SMA50\'] = data[\'Close\'].rolling(window=50).mean()\ndata[\'SMA200\'] = data[\'Close\'].rolling(window=200).mean()\n\n# Generate signals\ndata[\'Signal\'] = 0\ndata[\'Signal\'][50:] = np.where(data[\'SMA50\'][50:] > data[\'SMA200\'][50:], 1, 0)\n\n# Generate trading orders\ndata[\'Position\'] = data[\'Signal\'].diff()\n\n# Strategy logic: Buy when SMA50 crosses above SMA200, Sell when it crosses below\nbuy_signals = data[data[\'Position\'] == 1]\nsell_signals = data[data[\'Position\'] == -1]\n\n# Output the signals\nprint("Buy Signals:\\n", buy_signals[[\'Close\', \'SMA50\', \'SMA200\']])\nprint("\\nSell Signals:\\n", sell_signals[[\'Close\', \'SMA50\', \'SMA200\']])\n```'

In [56]:
sen = ""
for i in response:
    if i == "ticker":
        print("ticker_symbol = ", end="")
    sen += i
    print(i, end="")

```python
# Stock ID for Tesla
ticker = "TSLA"

import yfinance as yf
import pandas as pd
import numpy as np

# Fetch historical data for TSLA
data = yf.download(ticker, period="1y", interval="1d")

# Calculate Moving Averages
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# Generate signals
data['Signal'] = 0
data['Signal'][50:] = np.where(data['SMA50'][50:] > data['SMA200'][50:], 1, 0)

# Generate trading orders
data['Position'] = data['Signal'].diff()

# Strategy logic: Buy when SMA50 crosses above SMA200, Sell when it crosses below
buy_signals = data[data['Position'] == 1]
sell_signals = data[data['Position'] == -1]

# Output the signals
print("Buy Signals:\n", buy_signals[['Close', 'SMA50', 'SMA200']])
print("\nSell Signals:\n", sell_signals[['Close', 'SMA50', 'SMA200']])
```

In [61]:
sen

'```python\n# Stock ID for Tesla\nticker = "TSLA"\n\nimport yfinance as yf\nimport pandas as pd\nimport numpy as np\n\n# Fetch historical data for TSLA\ndata = yf.download(ticker, period="1y", interval="1d")\n\n# Calculate Moving Averages\ndata[\'SMA50\'] = data[\'Close\'].rolling(window=50).mean()\ndata[\'SMA200\'] = data[\'Close\'].rolling(window=200).mean()\n\n# Generate signals\ndata[\'Signal\'] = 0\ndata[\'Signal\'][50:] = np.where(data[\'SMA50\'][50:] > data[\'SMA200\'][50:], 1, 0)\n\n# Generate trading orders\ndata[\'Position\'] = data[\'Signal\'].diff()\n\n# Strategy logic: Buy when SMA50 crosses above SMA200, Sell when it crosses below\nbuy_signals = data[data[\'Position\'] == 1]\nsell_signals = data[data[\'Position\'] == -1]\n\n# Output the signals\nprint("Buy Signals:\\n", buy_signals[[\'Close\', \'SMA50\', \'SMA200\']])\nprint("\\nSell Signals:\\n", sell_signals[[\'Close\', \'SMA50\', \'SMA200\']])\n```'

In [59]:
python_repl = PythonREPL()

In [60]:
python_repl.run(sen)

[*********************100%***********************]  1 of 1 completed
<string>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'Buy Signals:\n Empty DataFrame\nColumns: [(Close, TSLA), (SMA50, ), (SMA200, )]\nIndex: []\n\nSell Signals:\n Empty DataFrame\nColumns: [(Close, TSLA), (SMA50, ), (SMA200, )]\nIndex: []\n'

In [4]:
import yfinance as yf
import pandas as pd

from langchain_experimental.utilities.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_core.tools import tool

# --- 1) Wrap your get_data function as a LangChain @tool ---
class MyTools:
    @tool
    def get_data(interval: str, period: str, ticker: str) -> pd.DataFrame:
        """Fetch historical stock data from Yahoo Finance."""
        try:
            df = yf.download(tickers=ticker, period=period, interval=interval)
        except Exception:
            df = pd.DataFrame(columns=[
                "Open","High","Low","Close","Adj Close","Volume"
            ])
        return df


    @tool
    def analyze(code: str) -> str:
        """
        Run arbitrary Python code (which can use the pandas DataFrame `df`)
        and return printed output.
        """
        return python_repl.run(code)

tools = [
    getattr(MyTools, fn)
    for fn in dir(MyTools)
    if callable(getattr(MyTools, fn)) and hasattr(getattr(MyTools, fn), "args_schema")
] 



In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",  # You can also use "gemini-1.5-pro" or "gemini-1.5-flash"
    temperature=0.7,
    google_api_key=os.getenv("genmini_api")
)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

ValueError: ZeroShotAgent does not support multi-input tool get_data.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",  # You can also use "gemini-1.5-pro" or "gemini-1.5-flash"
    temperature=0.7,
    google_api_key=os.getenv("genmini_api")
)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

plan = """
1) generate the strategy for tsla 
2) print the strategy code
2) get the data using the get_data function
3) run the code using the analyze function please also include the data you get from get_data function
4) print the result of the code
5) return the result of the code
# inside this REPL we have `df` available from step 1"""

print(agent.run(plan))



> Entering new AgentExecutor chain...


[*********************100%***********************]  1 of 1 completed


Invoking: `get_data` with `{'interval': '1d', 'period': '1mo', 'ticker': 'TSLA'}`


Price            Close        High         Low        Open     Volume
Ticker            TSLA        TSLA        TSLA        TSLA       TSLA
Date                                                                 
2025-03-24  278.390015  278.640015  256.329987  258.079987  169079900
2025-03-25  288.140015  288.200012  271.279999  283.600006  150361500
2025-03-26  272.059998  284.899994  266.510010  282.660004  153629800
2025-03-27  273.130005  291.850006  271.820007  272.480011  162572100
2025-03-28  263.549988  276.100006  260.570007  275.579987  123809400
2025-03-31  259.160004  260.559998  243.360001  249.309998  134008900
2025-04-01  268.459991  277.450012  259.250000  263.799988  146486900
2025-04-02  282.760010  284.989990  251.270004  254.600006  212787800
2025-04-03  267.279999  276.299988  261.510010  265.290009  136174300
2025-04-04  239.429993  261.000000  236.000000  255.380005  181229400
2025-


Invoking: `analyze` with `{'code': "import pandas as pd\n\n# Generate a simple moving average crossover strategy\n# Buy when the short-term moving average crosses above the long-term moving average\n# Sell when the short-term moving average crosses below the long-term moving average\n\ndef generate_strategy(df):\n    # Calculate moving averages\n    df['SMA_5'] = df['Close'].rolling(window=5).mean()\n    df['SMA_10'] = df['Close'].rolling(window=10).mean()\n\n    # Generate signals\n    df['Signal'] = 0.0\n    df['Signal'][df['SMA_5'] > df['SMA_10']] = 1.0\n    df['Signal'][df['SMA_5'] < df['SMA_10']] = -1.0\n\n    # Generate trading orders\n    df['Position'] = df['Signal'].diff()\n\n    # Buy signal\n    buy_signals = df[df['Position'] == 2]\n    # Sell signal\n    sell_signals = df[df['Position'] == -2]\n\n    return buy_signals, sell_signals\n\n# Generate strategy signals\nbuy_signals, sell_signals = generate_strategy(df)\n\nbuy_signals, sell_signals"}`


NameError("name 'df' is n

<string>:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<string>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

The strategy code has been generated and executed. Here are the buy and sell signals based on the moving average crossover strategy for TSLA:

Buy signals:
- Date: 2025-04-01
- Date: 2025-04-09

Sell signals:
- Date: 2025-04-07
- Date: 2025-04-17

These signals indicate potential entry and exit points for trading TSLA based on the simple moving average crossover strategy.

> Finished chain.
The strategy code has been generated and executed. Here are the buy and sell signals based on the moving average crossover strategy for TSLA:

Buy signals:
- Date: 2025-04-01
- Date: 2025-04-09

Sell signals:
- Date: 2025-04-07
- Date: 2025-04-17

These signals indicate potential entry and exit points for trading TSLA based on the simple moving average crossover strategy.
